In [1]:
import numpy as np
import scipy.signal

In [2]:
# The azimuth angles (in degrees) of the loudspeakers.
# Angles must be in counter-clockwise order.
emitter_angles = np.array([30, 110, 150, -150, -110, -30])
emitter_channel_ordering = np.array([0, 4, 6, 7, 5, 1])

In [3]:
# Unit vectors for the loudspeakers. Not that the matrix is constructed as the transpose of the vectors.
# When P' = G * L, the result of P' is the correct p = g1 L1 + g2 L2 by scaled vector addition.
# The projections of the unit vectors sum to the unit vector in the direction of the source.
L = np.array((np.cos(np.radians(emitter_angles)), np.sin(np.radians(emitter_angles)))).T

In [4]:
# Create successive pairs of speaker angles until looping around. The pair (n1, n2) where 
# n1 is 'more clock-wise' than n2.
panning_pairs = np.vstack([(i, i + 1) for i in range(len(emitter_angles) - 1)] + [(len(emitter_angles) - 1, 0)])

In [5]:
# The azimuth angles (in degrees) of the sources.
source_angles = np.array([0, 70, 130, 180, -130, -70])

In [6]:
# Calculate the cartesian directional unit vectors for the sound sources.
P = np.array((np.cos(np.radians(source_angles)), np.sin(np.radians(source_angles))))

In [7]:
# G is the matrix that contains the calculated gains between each emitter within
# a panning pair of emitters, for each emitter.
#
# For M sound sources, for N panning pairs, [n1, n2] where n1 is 'more clock-wise' than n2.
G = np.zeros((len(source_angles), len(emitter_angles), 2))

In [8]:
# For each source, calculate the gains matrix for all panning pairs.

# The directional unit vectors for a specific panning pair.
Ln1n2 = np.zeros((2, 2))

for m in range(len(source_angles)):
    pT = P[:, m].reshape(1, 2)
    for n in range(len(emitter_angles)):
        panning_pair = list(panning_pairs[n])
        Ln1n2[:,:] = L[panning_pair,:]
        
        G[m, n, :] = pT @ np.linalg.inv(Ln1n2)

        # Normalize the gains
        G[m, n, :] /= np.linalg.norm(G[m, n, :])

In [ ]:
class PannedSource:
    def __init__(self, source_angle, group, gains):
        self.source_angle = source_angle
        self.panning_pair = group
        self.gains = gains
        self.gains[gains < 0.000001] = 0.0
        self.panned_audio = None
        

    def __str__(self):
        return f'{self.source_angle} = {self.gains[0]} * {self.panning_pair[0]} + {self.gains[1]} * {self.panning_pair[1]}'
    
    def pan(self, source, input):
        '''
        source.shape == (m, 1)
        input.shape == (m, n) where m is the number of samples and n is the number of channels/speakers.

        Returns an m x n array with panned source correctly mixed into the appropriate channels of input.
        Specifically, 
            output[:,self.group] = self.gains * source + input[self.group]
        '''

        self.panned_audio = self.gains * np.repeat(source, 2).reshape(len(source), 2)
        return self.panned_audio

panned_sources = [PannedSource(source_angle=source_angles[m],
                                group=panning_pairs[n],
                                gains=G[m, n, :]) 
                                for m in range(len(source_angles)) 
                                for n in range(len(emitter_angles)) 
                                if ((G[m,n,0] == 0.0) and (G[m,n,1] == 1.0)) or np.logical_and(G[m,n,:] > 0, G[m,n,:] < 1).all()]

for (panned_source_index, ps) in enumerate(panned_sources):
    print(f'Source {panned_source_index:2} at {ps.source_angle:4} := {ps.gains[0]:0.4f} * {ps.panning_pair[0]:2}/{emitter_angles[ps.panning_pair[0]]:4} + {ps.gains[1]:0.4f} * {ps.panning_pair[1]:2}/{emitter_angles[ps.panning_pair[1]]:4}')

In [ ]:
# Extract panning pairs and gains from panned_sources
panned_pairs_gains = np.array([list(zip(panned_source.panning_pair, panned_source.gains)) for panned_source in panned_sources])

# Flatten the array and separate pairs and gains
panned_source_emitter_indices = panned_pairs_gains[:,:,0].astype(int).flatten()
panned_source_emitter_gains = panned_pairs_gains[:,:,1].flatten()
np.bincount(panned_source_emitter_indices, weights=panned_source_emitter_gains, minlength=len(emitter_angles))

In [11]:
fs = 48000
T = 10
t = np.linspace(0, T, fs * T)

frequencies = np.array([261.63, 329.63, 392.00, 523.25, 659.25, 783.99])

sources = 1 / source_angles.size * np.sin(2 * np.pi * frequencies[:len(source_angles)][:,np.newaxis] * t)
#sourcesT = sources.T
#sources = np.ones((len(source_angles), len(t)))

In [12]:
panned_output = np.zeros((fs * T, len(emitter_angles)))

for (panned_source_index, panned_source) in enumerate(panned_sources):
    panned_output[:,panned_source.panning_pair] += panned_source.pan(sources[:,panned_source_index], panned_output)

# Collect all of the panning_pairs from the panned sources.
panned_sources_pairs = [panned_source.panning_pair for panned_source in panned_sources]

# Flatten the list of pairs to give the indices of the emitters that had panned sources added to them.
panned_source_emitters = np.unique([emitter for pair in panned_sources_pairs for emitter in pair])
panned_source_channel_indices = [emitter_channel_ordering[emitter] for emitter in panned_source_emitters]

In [13]:

output = np.zeros((T * fs, 8))

# First write the unpanned sources to a wav file but shuffle the indexing of the sources so that
# no audio is written to the output channel indices 2 and 3.
unassigned_channels = [i for i in range(output.shape[1]) if i not in [2, 3]]

for panned_source_index, source in enumerate(sources):
    if panned_source_index < len(unassigned_channels):
        output[:, unassigned_channels[panned_source_index]] = source

scipy.io.wavfile.write('unassigned_sources.wav', fs, output.astype(np.float32))


In [14]:
output = np.zeros((T * fs, 8))

# Add the panned sources to the output.
for (emitter_index, channel_index) in enumerate(emitter_channel_ordering):
    output[:,channel_index] += panned_output[:,emitter_index]

MINUS_3_DB = 10.0**(-3.0/20.0)
output[:, panned_source_channel_indices] *= MINUS_3_DB
        
# Write output as a 32-bit float WAV file.
scipy.io.wavfile.write('output.wav', fs, output.astype(np.float32))
